# Personalities


## Setup


In [1]:
import pandas as pd
from io import StringIO
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import (
    CommaSeparatedListOutputParser,
)

In [2]:
openai_client = OpenAI()
openai_chat = ChatOpenAI(model="gpt-4-turbo", temperature=0)
csv_parser = CommaSeparatedListOutputParser()
csv_format_instructions = csv_parser.get_format_instructions()

## Datasets


In [3]:
personalities_df = pd.read_csv(
    "./datasets/personalities.csv",
    delimiter="|",
    skipinitialspace=True,
)

In [4]:
personality_data = personalities_df[["TestName", "PersonalityType"]].to_csv(sep=",")
personality_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"You are an expert on personality types. I will send messages about the following personality information. You will provide helpful responses.\n\npersonalities:\n```\n{personality_data}\n```",
        ),
        ("user", "{input}\n\nformat_instructions:{format_instructions}"),
    ]
)
personality_csv_chain = personality_prompt | openai_chat | csv_parser

In [5]:
try:
    leadership_df = pd.read_csv("./datasets/leadership_styles.csv", sep=",")
except FileNotFoundError:
    leadership_result = personality_csv_chain.invoke(
        {
            "input": "Provide me with the leadership style that each PersonalityType prefers to use. In addition to the leadership style, give short and concise 2 sentence descriptions.",
            "format_instructions": f"{csv_format_instructions}\n\n include response column names: PersonalityType, LeadershipStyle, LeadershipDescription",
        }
    )
    leadership_df = pd.read_csv(StringIO("".join(leadership_result)), delimiter=",")
    leadership_df.to_csv("./datasets/leadership_styles.csv", sep=",", index=False)

In [6]:
try:
    communication_df = pd.read_csv("./datasets/communication_styles.csv", sep=",")
except FileNotFoundError:
    communication_result = personality_csv_chain.invoke(
        {
            "input": "Provide me with the communication style that each PersonalityType prefers to use. In addition to the communication style, give short and concise 2 sentence descriptions.",
            "format_instructions": f"{csv_format_instructions}\n\n include response column names: PersonalityType, CommunicationStyle, CommunicationDescription",
        }
    )
    communication_df = pd.read_csv(
        StringIO("".join(communication_result)), delimiter=","
    )
    communication_df.to_csv("./datasets/communication_styles.csv", sep=",", index=False)

In [7]:
try:
    motivators_df = pd.read_csv("./datasets/motivators.csv", sep=",")
except FileNotFoundError:
    motivators_result = personality_csv_chain.invoke(
        {
            "input": "Provide me with the motivators for each PersonalityType. Separate each motivator with a '|' character.",
            "format_instructions": f"{csv_format_instructions}\n\n include response column names: PersonalityType, Motivators",
        }
    )
    motivators_df = pd.read_csv(StringIO("".join(motivators_result)), delimiter=",")
    motivators_df.to_csv("./datasets/motivators.csv", sep=",", index=False)

In [8]:
try:
    fictional_characters_df = pd.read_csv(
        "./datasets/fictional_characters.csv", sep=","
    )
except FileNotFoundError:
    fictional_characters_result = personality_csv_chain.invoke(
        {
            "input": "Provide me with fictional characters for each PersonalityType. Separate each fictional character with a '|' symbol.",
            "format_instructions": f"{csv_format_instructions}\n\n include response column names: PersonalityType, FictionalCharacters",
        }
    )
    fictional_characters_df = pd.read_csv(
        StringIO("".join(fictional_characters_result)), delimiter=","
    )
    fictional_characters_df.to_csv(
        "./datasets/fictional_characters.csv", sep=",", index=False
    )

## Main


In [9]:
personalities_df = personalities_df.merge(leadership_df, on="PersonalityType")
personalities_df = personalities_df.merge(communication_df, on="PersonalityType")
personalities_df = personalities_df.merge(motivators_df, on="PersonalityType")
personalities_df = personalities_df.merge(fictional_characters_df, on="PersonalityType")

personalities_df

,TestName,PersonalityType,Archetype,Description,LeadershipStyle,LeadershipDescription,CommunicationStyle,CommunicationDescription,Motivators,FictionalCharacters
0,Myers-Briggs,ISTJ,Inspector,"Responsible organizers, driven to create order...",Directive,ISTJs prefer a structured and orderly approach...,Direct and Detailed,ISTJs prefer clearstraightforward communicatio...,Responsibility|Tradition|Stability|Order,Eddard Stark (Game of Thrones) | Hermione Gran...
1,Myers-Briggs,ISFJ,Protector,"Dedicated caretakers, eager to help and provid...",Supportive,ISFJs lead with a focus on harmony and coopera...,Warm and Considerate,ISFJs communicate in a warmkind manneroften fo...,Harmony|Safety|Stability|Service to others,Samwise Gamgee (Lord of the Rings) | Dr. John ...
2,Myers-Briggs,INFJ,Counselor,"Insightful visionaries, seeking meaning in rel...",Transformative,INFJs are visionary leaders who aim to inspire...,Insightful and Conceptual,INFJs enjoy deepmeaningful conversations about...,Meaningful work|Personal growth|Integrity|Cont...,Jon Snow (Game of Thrones) | Aragorn (Lord of ...
3,Myers-Briggs,INTJ,Mastermind,"Strategic thinkers, determined to improve syst...",Strategic,INTJs prefer a leadership style that is analyt...,Strategic and Logical,INTJs prefer communication that is logical and...,Competence|Independence|Innovation|Achievement,Bruce Wayne (Batman) | Gandalf (Lord of the Ri...
4,Myers-Briggs,ISTP,Craftsman,"Explorers of the physical world, adept at unde...",Adaptive,ISTPs are flexible and pragmatic leadersoften ...,Concise and Practical,ISTPs communicate in a straightforward and act...,Practical results|Freedom|Variety|Hands-on exp...,James Bond (James Bond) | Han Solo (Star Wars)...
5,Myers-Briggs,ISFP,Composer,"Gentle caretakers, living in the moment and en...",Authentic,ISFPs lead by creating an open and accepting e...,Empathetic and Gentle,ISFPs have a softunderstanding communication s...,Aesthetic experiences|Harmony|Flexibility|Gent...,Frodo Baggins (Lord of the Rings) | Luke Skywa...
6,Myers-Briggs,INFP,Healer,"Idealistic dreamers, guided by core values and...",Empathetic,INFPs lead with a focus on values and integrit...,Idealistic and Expressive,INFPs communicate in an idealisticpassionate w...,Idealism|Harmony|Authenticity|Exploring inner ...,Luke Skywalker (Star Wars) | Frodo Baggins (Lo...
7,Myers-Briggs,INTP,Architect,"Inventive thinkers, curious about the world. S...",Innovative,INTPs prefer a leadership style that fosters i...,Analytical and Abstract,INTPs enjoy discussions about complex concepts...,Intellectual challenge|Competence|Independence...,Spencer Reid (Criminal Minds) | Sherlock Holme...
8,Myers-Briggs,ESTP,Dynamo,"Energetic thrill-seekers, thrive on action. Pr...",Dynamic,ESTPs are energetic and action-oriented leader...,Dynamic and Direct,ESTPs are straightforward and action-oriented ...,Action|Impact|Excitement|Problem-solving,Tyler Durden (Fight Club) | Jack Sparrow (Pira...
9,Myers-Briggs,ESFP,Performer,"Spontaneous entertainers, enjoying the spotlig...",Inspirational,ESFPs lead with charisma and enthusiasmoften c...,Enthusiastic and Expressive,ESFPs communicate in a livelyengaging mannerof...,Positive interactions|Fun|Spontaneity|Aestheti...,Peter Pan (Peter Pan) | Ron Weasley (Harry Pot...
